In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [ ]:
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# CIFAR-100 dataset has 100 classes
num_classes = 100

In [ ]:
# Image Preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalization
])

In [ ]:
# CIFAR-100 data
train_dataset = datasets.CIFAR100(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR100(root='./data', train=False, transform=transform)

100%|██████████| 169001437/169001437 [00:03<00:00, 46707707.14it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


In [ ]:
# Data Loader
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


In [ ]:
# Define a GoogLeNet model for CIFAR-10
class GoogLeNetCIFAR100(models.GoogLeNet):
    def __init__(self, num_classes=100):
        super(GoogLeNetCIFAR100, self).__init__(num_classes=num_classes, aux_logits=False, init_weights=True)
        # Redefine the first convolutional layer to accept CIFAR-10's 32x32 pixel images
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # Follow the forward pass of the GoogLeNet model
        return super(GoogLeNetCIFAR100, self).forward(x)


In [ ]:
# Instantiate the model and transfer it to the GPU
model = GoogLeNetCIFAR100(num_classes=num_classes).to(device)


In [ ]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Function to train the model
def train(model, criterion, optimizer, train_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            # Transfer inputs and labels to the GPU
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100}')
                running_loss = 0.0

In [ ]:
# Function to test the model
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            # Transfer images and labels to the GPU
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy


In [ ]:
# Start training
train(model, criterion, optimizer, train_loader, epochs=10)



Epoch 1, Batch 100, Loss: 4.516444692611694
Epoch 1, Batch 200, Loss: 4.238213219642639
Epoch 1, Batch 300, Loss: 4.1268888235092165
Epoch 1, Batch 400, Loss: 4.02144907951355
Epoch 1, Batch 500, Loss: 3.9263324189186095
Epoch 1, Batch 600, Loss: 3.830439863204956
Epoch 1, Batch 700, Loss: 3.7423854541778563
Epoch 2, Batch 100, Loss: 3.594779953956604
Epoch 2, Batch 200, Loss: 3.5031527400016786
Epoch 2, Batch 300, Loss: 3.432232382297516
Epoch 2, Batch 400, Loss: 3.393256375789642
Epoch 2, Batch 500, Loss: 3.28625926733017
Epoch 2, Batch 600, Loss: 3.202833218574524
Epoch 2, Batch 700, Loss: 3.152721028327942
Epoch 3, Batch 100, Loss: 3.0173394083976746
Epoch 3, Batch 200, Loss: 2.9532887840271
Epoch 3, Batch 300, Loss: 2.889402515888214
Epoch 3, Batch 400, Loss: 2.854636006355286
Epoch 3, Batch 500, Loss: 2.7945648670196532
Epoch 3, Batch 600, Loss: 2.755086920261383
Epoch 3, Batch 700, Loss: 2.698475317955017
Epoch 4, Batch 100, Loss: 2.54119198679924
Epoch 4, Batch 200, Loss: 2.500

In [ ]:
# Test the model
accuracy = test(model, test_loader)
print(f'Accuracy of the model on the 10000 test images: {accuracy}%')


Accuracy of the model on the 10000 test images: 49.7%
